<a href="https://colab.research.google.com/github/Noman-494/Deep_learning/blob/main/Movie_assigment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.text import tokenizer_from_json
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [2]:
# Load dataset
max_features = 10000  # Vocabulary size
maxlen = 200  # Sequence length to pad
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
word_index = imdb.get_word_index()
reverse_word_index = {v: k for k, v in word_index.items()}
# Show 5 samples
def decode_review(sequence):
    return " ".join([reverse_word_index.get(i - 3, "?") for i in sequence])  # Offset by 3 for reserved indices

print("First 5 Reviews in Words:")
for i in range(5):
    print(f"Review {i+1}: {decode_review(x_train[i])}")

1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
First 5 Reviews in Words:
Review 1: ? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but t

In [4]:
# Tokenization (pre-vectorized by IMDB, so skip tokenization)

# Pad sequences
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)


In [5]:
model = Sequential([
    Embedding(input_dim=max_features, output_dim=128, input_length=maxlen),
    LSTM(64, return_sequences=False),
    Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [6]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print("Training the model...")
history = model.fit(x_train, y_train, epochs=5, batch_size=32, validation_split=0.2)


Training the model...
Epoch 1/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 87s 135ms/step - accuracy: 0.7114 - loss: 0.5372 - val_accuracy: 0.7828 - val_loss: 0.4534
Epoch 2/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 84s 134ms/step - accuracy: 0.8831 - loss: 0.2933 - val_accuracy: 0.8698 - val_loss: 0.3046
Epoch 3/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 83s 133ms/step - accuracy: 0.9262 - loss: 0.1937 - val_accuracy: 0.8562 - val_loss: 0.3498
Epoch 4/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 142s 133ms/step - accuracy: 0.9532 - loss: 0.1322 - val_accuracy: 0.8360 - val_loss: 0.4355
Epoch 5/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 142s 133ms/step - accuracy: 0.9607 - loss: 0.1086 - val_accuracy: 0.8662 - val_loss: 0.4434


In [7]:
# Evaluate model
print("Evaluating the model...")
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Evaluating the model...
782/782 ━━━━━━━━━━━━━━━━━━━━ 28s 36ms/step - accuracy: 0.8562 - loss: 0.4603
Test Accuracy: 85.97%


In [8]:
# Predict sentiment of a review
sample_review = decode_review(x_test[0])  # Decode a test review
sample_review_vectorized = pad_sequences([x_test[0]], maxlen=maxlen)  # Vectorize
predicted_sentiment = model.predict(sample_review_vectorized)
print(f"Sample Review: {sample_review}")
print(f"Predicted Sentiment: {'Positive' if predicted_sentiment[0][0] > 0.5 else 'Negative'}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
Sample Review: ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? please give this one a miss br br ? ? and the rest of the cast rendered terrible performances the show is flat flat flat br br i don't know how michael madison could have allowed this one on his plate he almost seemed to know this wasn't going to work out and his performance was quite ? so all you madison fans give this a miss
Predicted Sentiment: Negative


In [9]:
# Predict sentiments for the first 5 samples
print("\nPredictions for the First 5 Test Samples:\n")
for i in range(5):
    # Decode the review
    review_text = decode_review(x_test[i])
    # Predict sentiment
    review_vectorized = pad_sequences([x_test[i]], maxlen=maxlen)
    predicted_sentiment = model.predict(review_vectorized, verbose=0)[0][0]
    predicted_label = 'Positive' if predicted_sentiment > 0.5 else 'Negative'
    # Actual label
    actual_label = 'Positive' if y_test[i] == 1 else 'Negative'

    # Display results
    print(f"Review {i+1}: {review_text[:100]}...")  # Truncated for readability
    print(f"Predicted Sentiment: {predicted_label} (Confidence: {predicted_sentiment:.2f})")
    print(f"Actual Sentiment: {actual_label}")
    print("-" * 80)



Predictions for the First 5 Test Samples:

Review 1: ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ...
Predicted Sentiment: Negative (Confidence: 0.05)
Actual Sentiment: Negative
--------------------------------------------------------------------------------
Review 2: psychological ? it's very interesting that robert altman directed this considering the style and str...
Predicted Sentiment: Positive (Confidence: 1.00)
Actual Sentiment: Positive
--------------------------------------------------------------------------------
Review 3: everyone's horror the ? promptly eats the mayor and then goes on a merry rampage ? citizens at rando...
Predicted Sentiment: Positive (Confidence: 0.67)
Actual Sentiment: Positive
--------------------------------------------------------------------------------
Review 4: ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? i generally love this type of movie however this time i foun...
Predicted Sentiment: Positive